In [1]:
!pip install AugmentedSocialScientist pandas numpy

  Using cached AugmentedSocialScientist-2.2.1.tar.gz (10.0 kB)
  Preparing metadata (setup.py) ... done
  Using cached pandas-2.2.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached scikit_learn-1.4.1.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached sentencepiece-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.3.0-py3-none-any.whl.metadata (13 kB)
  Using cached huggingface_hub-0.21.4-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2023.12.25-cp311-cp311-manylinux

In [2]:
import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from torch import cuda

cuda.get_device_name(0)

'Tesla T4'

In [3]:
import pandas as pd

annotations_file = pd.read_csv("../data/intermediate/current_annotation_file.csv")
annotations_file.sample(5)

,sentence_id,Personne,sentence_text
1060,760003,NaN,"En cas de défaillance ou de négligence, ils\n\..."
1191,1281052,NaN,"De fait, il existe d’assez nombreuses disposit..."
210,1200528,Personne relationnelle,Mais\t\n s’il\t\n existe\t\n un\t\n droit\...
1174,1280008,NaN,Avis \n \n1. Comment contribuer au changement...
1126,1330246,NaN,"Il apparait important, quelle que soit la r..."


In [4]:
annotations = annotations_file.dropna()
annotations.sample(5)

,sentence_id,Personne,sentence_text
353,850041,Personne à protéger,Le risque n’est plus envisagé dans sa vision \...
406,140226,Agent responsable,Il est possible\net peutêtre souhaitable d'all...
320,580308,Personne relationnelle,"Autant que cela n'est\npas établi, il doit inf..."
170,1291599,Individu,Cette loi rend \nles directives anticipées con...
352,780133,Agent responsable,Cet avis était accompagné d’un rapport rappela...


In [5]:
annotations["Personne"].value_counts()

Personne
Personne à protéger       143
Matériel biologique       127
Individu                  118
Agent responsable         104
Personne relationnelle     97
Genre/espèce humaine       90
Humain comme empathie      10
Poubelle                    8
Name: count, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
 
train_data, test_data = train_test_split(annotations, train_size=0.67)

In [7]:
from AugmentedSocialScientist.models import Camembert

bert = Camembert()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


There are 1 GPU(s) available.
We will use GPU 0: Tesla T4


In [8]:
batch_size = 20
train_loader = bert.encode(train_data.sentence_text.values,
train_data.Personne.values, batch_size=batch_size)

  0%|          | 0/466 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (620 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/466 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}


In [9]:
test_loader = bert.encode(test_data.sentence_text.values,
test_data.Personne.values, batch_size=batch_size)

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}


In [10]:
scores = bert.run_training(
    train_loader,
    test_loader,
    n_epochs=5,
    lr=2e-5,
    random_state=1968,
    save_model_as='clickbait'
    )

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/mamba/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 5 ========
Training...

  Average training loss: 2.02
  Training took: 0:00:38

Running Validation...

  Average test loss: 1.97
  Validation took: 0:00:05
                        precision    recall  f1-score   support

     Agent responsable       0.00      0.00      0.00        31
  Genre/espèce humaine       0.00      0.00      0.00        28
 Humain comme empathie       0.00      0.00      0.00         5
              Individu       0.00      0.00      0.00        39
   Matériel biologique       0.53      0.67      0.59        42
Personne relationnelle       0.00      0.00      0.00        35
   Personne à protéger       0.22      0.87      0.35        45
              Poubelle       0.00      0.00      0.00         6

              accuracy                           0.29       231
             macro avg       0.09      0.19      0.12       231
          weighted avg       0.14      0.29      0.18       231


======== Epoch 2 / 5 ========
Training...


/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/m


  Average training loss: 1.94
  Training took: 0:00:40

Running Validation...

  Average test loss: 1.92
  Validation took: 0:00:06
                        precision    recall  f1-score   support

     Agent responsable       0.00      0.00      0.00        31
  Genre/espèce humaine       0.00      0.00      0.00        28
 Humain comme empathie       0.00      0.00      0.00         5
              Individu       0.00      0.00      0.00        39
   Matériel biologique       0.54      0.83      0.65        42
Personne relationnelle       0.00      0.00      0.00        35
   Personne à protéger       0.24      0.89      0.38        45
              Poubelle       0.00      0.00      0.00         6

              accuracy                           0.32       231
             macro avg       0.10      0.22      0.13       231
          weighted avg       0.14      0.32      0.19       231


======== Epoch 3 / 5 ========
Training...


...

/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/m


  Average training loss: 1.84
  Training took: 0:00:42

Running Validation...

  Average test loss: 1.84
  Validation took: 0:00:06
                        precision    recall  f1-score   support

     Agent responsable       0.00      0.00      0.00        31
  Genre/espèce humaine       0.00      0.00      0.00        28
 Humain comme empathie       0.00      0.00      0.00         5
              Individu       0.00      0.00      0.00        39
   Matériel biologique       0.62      0.90      0.74        42
Personne relationnelle       0.00      0.00      0.00        35
   Personne à protéger       0.22      0.84      0.35        45
              Poubelle       0.00      0.00      0.00         6

              accuracy                           0.33       231
             macro avg       0.11      0.22      0.14       231
          weighted avg       0.16      0.33      0.20       231


======== Epoch 5 / 5 ========
Training...


/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/m


  Average training loss: 1.81
  Training took: 0:00:42

Running Validation...

  Average test loss: 1.83
  Validation took: 0:00:06
                        precision    recall  f1-score   support

     Agent responsable       0.00      0.00      0.00        31
  Genre/espèce humaine       0.00      0.00      0.00        28
 Humain comme empathie       0.00      0.00      0.00         5
              Individu       1.00      0.03      0.05        39
   Matériel biologique       0.65      0.95      0.77        42
Personne relationnelle       0.00      0.00      0.00        35
   Personne à protéger       0.23      0.84      0.36        45
              Poubelle       0.00      0.00      0.00         6

              accuracy                           0.34       231
             macro avg       0.23      0.23      0.15       231
          weighted avg       0.33      0.34      0.22       231


Training complete!


/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.11/site-packages/sklearn/m

In [11]:
import numpy as np
pred_data = annotations_file[annotations_file.isna().any(axis=1)].drop('Personne', axis=1).dropna()
pred_loader = bert.encode(pred_data.sentence_text.values, batch_size=batch_size)
pred_proba = bert.predict_with_model(pred_loader, model_path='./models/clickbait')
pred_data['pred_label'] = np.argmax(pred_proba, axis=1)
pred_data['pred_proba'] = np.max(pred_proba, axis=1)

  0%|          | 0/798 [00:00<?, ?it/s]

  0%|          | 0/798 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}


In [12]:
pred_data

,sentence_id,sentence_text,pred_label,pred_proba
700,1220587,S’il semble dès lors naïf de penser que le sen...,6,0.195131
701,30245,Personne ne soutient qu'il soit possible de to...,4,0.242023
702,880051,Le temps de parole doit être suffisant pour \n...,6,0.196267
703,1120305,»\n24 Avis N°107 du 15 octobre 2009 sur les p...,4,0.258472
704,600333,Sur la question du champ d'application de la l...,4,0.171090
...,...,...,...,...
1495,940495,La garde à vue est une mesure privative de lib...,6,0.195310
1496,1200490,50\t\n “Il\t\n convient\t\n surtout\t\n de...,4,0.197915
1497,1400183,Cette mosaïque de situ ations d’inégalité s en...,6,0.195957
1498,670025,Dans le souci de clarification exposé plus hau...,4,0.257691
